In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
df = pd.read_csv('../data/preprocessed_train.csv', encoding='utf-8')
df.head()

,title,author,text,label,removed_punc,tokens,filtered_tokens,clean_tokens,lemma_words,clean_text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide We Didn’t Even See Comey’s Lett...,"['house', 'dem', 'aide', 'we', 'didn’t', 'even...","['house', 'aide', 'didn’t', 'even', 'comey’s',...","['house', 'aide', 'didn’t', 'even', 'comey’s',...","['house', 'aide', 'didn’t', 'even', 'comey’s',...",house aide didn’t even comey’s letter jason ch...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,Ever get the feeling your life circles the rou...,"['ever', 'get', 'the', 'feeling', 'your', 'lif...","['ever', 'feeling', 'your', 'life', 'circles',...","['ever', 'feeling', 'life', 'circles', 'rounda...","['ever', 'feeling', 'life', 'circle', 'roundab...",ever feeling life circle roundabout rather hea...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired October 29 2...,"['why', 'the', 'truth', 'might', 'get', 'you',...","['truth', 'might', 'fired', 'october', '2016',...","['truth', 'might', 'fired', 'october', '2016',...","['truth', 'might', 'fired', 'october', '2016',...",truth might fired october 2016 tension intelli...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Videos 15 Civilians Killed In Single US Airstr...,"['videos', '15', 'civilians', 'killed', 'in', ...","['videos', 'civilians', 'killed', 'single', 'a...","['videos', 'civilians', 'killed', 'single', 'a...","['video', 'civilian', 'killed', 'single', 'air...",video civilian killed single airstrike identif...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Print \nAn Iranian woman has been sentenced to...,"['print', 'an', 'iranian', 'woman', 'has', 'be...","['print', 'iranian', 'woman', 'been', 'sentenc...","['print', 'iranian', 'woman', 'sentenced', 'ye...","['print', 'iranian', 'woman', 'sentenced', 'ye...",print iranian woman sentenced year prison iran...


In [88]:
data = df.loc[df['label'] == 1, 'clean_text'][:50]
test = df.loc[df['label'] == 1, 'clean_text'][6000:]
text_list = data.tolist()
test = test.tolist()
text_list= [text for text in text_list if not (isinstance(text, float) and np.isnan(text))]
test= [text for text in test if not (isinstance(text, float) and np.isnan(text))]
#del text_list[7406] #este indice es un string con palabras en ruso


# Generator with BERT

### Tokenization

In [9]:
from transformers import BertTokenizer, BertForMaskedLM

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
from transformers import AdamW
# Tokenize your dataset
max_length = 512
tokenized_dataset = tokenizer(text_list, truncation=True, max_length=max_length, return_tensors='pt')

# Define the device
device = torch.device('cpu')

# Get the input IDs and attention mask tensors from the tokenized dataset
input_ids = tokenized_dataset['input_ids']
attention_mask = tokenized_dataset['attention_mask']

# Create a TensorDataset object from the input_ids and attention_mask tensors
dataset = torch.utils.data.TensorDataset(input_ids, attention_mask)
#input_ids = input_ids.to(device)
#attention_mask = attention_mask.to(device)

# Define the optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=5e-5)

# Create data loaders
train_loader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=True)

d:\programas\programas\github repos\nlp\nlp\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Model training

In [11]:
# Define your training loop
num_epochs = 3
model.train()
for epoch in range(num_epochs):
    for batch in train_loader:

        # Get the input and target sequences
        input_ids = batch[0]
        attention_mask = batch[1]

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)

        # Compute the loss
        loss = outputs.logits.view(-1, outputs.logits.shape[-1]).float().to(device)
        labels = input_ids.view(-1).to(device)
        loss = torch.nn.functional.cross_entropy(loss, labels)

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Zero the gradients
        optimizer.zero_grad()

    print(f'Epoch {epoch+1} loss: {loss.item()}')

Epoch 1 loss: 1.5594005584716797
Epoch 2 loss: 0.35310813784599304
Epoch 3 loss: 0.19861313700675964


### Model Output

In [78]:
# Define the prompt
prompt = "Argentina imposes "

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors='pt')

# Generate text
max_length = 12
output = model.generate(input_ids=input_ids, max_length=max_length, repetition_penalty=1.6)

# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

argentina imposes quentin garrett ’ lethal suspension revelationlay niall


In [52]:
# Define the prompt
prompt = "Donald Trump"
# Generate text
max_length = 6

for _ in range(10):
    # Tokenize the generated text
    input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors='pt')

    # Generate text
    output = model.generate(input_ids=input_ids, max_length=max_length, repetition_penalty=1.5)

    # Decode the generated text
    prompt = tokenizer.decode(output[0], skip_special_tokens=True)

    max_length += 3 


print(prompt)

donald trumprockieu ’ ” conclusion quentin suspensionguelay niall patrice notwithstanding “ rwanda thanksgiving sava brynweelmanibly genocide countlessbh din kellan civilian resultith nialliling


# Generator with Transformers

In [4]:
text_data = text_list[:6000]

### tokenization of data

In [6]:
from transformers import AdamW, GPT2LMHeadModel, GPT2Tokenizer

# Tokenize the dataset
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenized_dataset = tokenizer.encode(text_data, add_special_tokens=True, max_length=1024, truncation=True)

# Convert the tokenized dataset to tensors
input_ids = torch.tensor(tokenized_dataset[:-1]).unsqueeze(0)
labels = torch.tensor(tokenized_dataset[1:]).unsqueeze(0)

dataset = torch.utils.data.TensorDataset(input_ids, labels)

# Create data loaders
train_loader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True)

# Load the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Fine-tune the model
optimizer = AdamW(model.parameters(), lr=1e-5)

model.train()

d:\programas\programas\github repos\nlp\nlp\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

### Model Training

In [7]:
for epoch in range(5):
    for batch in train_loader:

        # Get the input and target sequences
        input_ids = batch[0]
        target_ids = batch[1]

        # Forward pass
        outputs = model(input_ids, labels=target_ids)

        # Compute the loss
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Zero the gradients
        optimizer.zero_grad()



### Model Output

Si se juega con los parametros ingresados en el metodo generate es posible cambiar la estructura de la generación de noticias. Revisar la documentación.

In [26]:
seed_text = "Argentina grows"
input_ids = tokenizer.encode(seed_text, return_tensors='pt')
output = model.generate(
    input_ids,
    max_length=300,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=2.0,
)
# Decode the generated sequences
generated_text = []
for sequence in output:
    sequence = sequence.tolist()
    text = tokenizer.decode(sequence, skip_special_tokens=True)
    generated_text.append(text)

print("Generated Fake News: ", generated_text)

Generated Fake News:  ['Argentina grows its own food.\nThe country\'s agriculture minister, Josef Kostas said that the government would not be able to provide a solution for farmers in this area because of lack and need from time-to: "We are going through an economic crisis." He added he was concerned about how it could happen with such large scale agricultural production as there is no way possible without using any kind or all methods at present," Mr.Kosovo told reporters on Wednesday (Nov 17).']


### Perplexity testing

In [87]:
test = text_list[6000:]
test

[]

In [91]:
test_set_tokens = tokenizer.encode(test, add_special_tokens=True, max_length=512, truncation=True)
# Convert the tokenized dataset to tensors
input_ids = torch.tensor(test_set_tokens[:-1]).unsqueeze(0)
labels = torch.tensor(test_set_tokens[1:]).unsqueeze(0)

In [92]:
loss = model(input_ids = input_ids, labels = labels).loss
ppl = torch.exp(loss)
print("Perplexity score: ", ppl)

Perplexity score:  tensor(818.2751, grad_fn=<ExpBackward0>)


perplexity gpt2: 1.4392; perplexity bert: 818